In [ ]:
import pandas as pd
from pathlib import Path
from datetime import datetime
import sys
import json
import os
import locale

sys.path.append('../scripts')
from utils import ensure_path_exists

# Função para carregar configurações
def load_settings(settings_file='../configs/settings.json'):
    """Carrega configurações de um arquivo JSON"""
    try:
        if os.path.exists(settings_file):
            with open(settings_file, 'r', encoding='utf-8') as f:
                settings = json.load(f)
                print(f"✅ Configurações carregadas de: {settings_file}")
                return settings
        else:
            print(f"⚠️ Arquivo {settings_file} não encontrado, usando configurações padrão")
            return {
                "api_url": "https://api.bcb.gov.br/dados/serie/bcdata.sjson?serie=12",
                "raw_data_path": "./data/raw/"
            }
    except Exception as e:
        print(f"❌ Erro ao carregar configurações: {e}")
        return {
            "api_url": "https://api.bcb.gov.br/dados/serie/bcdata.sjson?serie=12",
            "raw_data_path": "./data/raw/"
        }

# Carrega configurações
settings = load_settings()
RAW_DATA_PATH = Path(settings["raw_data_path"])
PROCESSED_PATH = Path(settings["processed_data_path"])
ensure_path_exists(PROCESSED_PATH)

# Encontra o último arquivo parquet salvo
parquet_files = sorted(RAW_DATA_PATH.glob("*.parquet"), reverse=True)
if not parquet_files:
    raise FileNotFoundError("Nenhum arquivo .parquet encontrado em data/raw")
file_path = parquet_files[0]
print(f"📥 Carregando arquivo: {file_path}")

# Lê o arquivo parquet
df = pd.read_parquet(file_path)
df['data'] = pd.to_datetime(df['data'])

# Limpeza e normalização
df = df.dropna()
df['valor'] = df['valor'].astype(int)

# Gera texto com contexto por linha
def gerar_contexto(row):
    locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')
    data_formatada = row['data'].strftime("%B de %Y")  # Ex: março de 2005
    valor_formatado = f"R$ {row['valor'] / 100:.2f}".replace('.', ',')
    return f"No mês de {data_formatada}, o valor da série era {valor_formatado}."

df['contexto'] = df.apply(gerar_contexto, axis=1)

# Salva os textos como CSV para vetorização posterior
output_file = PROCESSED_PATH / "dados_contextualizados.csv"
df[['data', 'valor', 'contexto']].to_csv(output_file, index=False)

print(f"✅ Dados processados salvos em: {output_file}")
print(df[['data', 'contexto']].head())


✅ Configurações carregadas de: ../configs/settings.json
📥 Carregando arquivo: data/raw/dados_bcb_20250718_233936.parquet
✅ Dados processados salvos em: data/processed/dados_contextualizados.csv
        data                                           contexto
0 1980-02-01  No mês de fevereiro de 1980, o valor da série ...
1 1980-03-01  No mês de março de 1980, o valor da série era ...
2 1980-04-01  No mês de abril de 1980, o valor da série era ...
3 1980-05-01  No mês de maio de 1980, o valor da série era R...
4 1980-06-01  No mês de junho de 1980, o valor da série era ...
